# TODO: berefore you run this notebook, start the PostgreSQL db service!

Load Data into PostgreSQL DB

In [1]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

DB_HOST = "localhost"
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_PORT = "5401"

In [10]:
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

try:
    # Connect to the db to ensure connection is working
    conn_test = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT
    )
    conn_test.close()
    print("Successfully connected to PostgreSQL database!")

    # Load DataFrames
    payments_df = pd.read_csv('Payments.csv')
    clients_df = pd.read_csv('Clients.csv')

    # IMPORTANT: Convert 'transaction_date' from EPOCH to datetime BEFORE loading to DB, in order to have it as a proper timestamp
    payments_df['transaction_date'] = pd.to_datetime(payments_df['transaction_date'], unit='s')


    # Write DataFrames to PostgreSQL tables
    # if_exists: 'replace' will drop the table if it exists and recreate it
    # index=False: Do not write the DataFrame index as a column in the database table

    print("Importing Payments DataFrame to 'payments' table...")
    payments_df.to_sql('payments', engine, if_exists='replace', index=False)
    print("Payments data imported successfully!")

    print("Importing Clients DataFrame to 'clients' table...")
    clients_df.to_sql('clients', engine, if_exists='replace', index=False)
    print("Clients data imported successfully!")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the engine connection (though SQLAlchemy manages connections in a pool)
    engine.dispose()

Successfully connected to PostgreSQL database!
Importing Payments DataFrame to 'payments' table...
Payments data imported successfully!
Importing Clients DataFrame to 'clients' table...
Clients data imported successfully!


Load Data from PostgreSQL DB

In [11]:
engine = create_engine(f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

try:
    # Read data back from PostgreSQL into pandas DataFrames
    payments_df_from_db = pd.read_sql('SELECT * FROM payments', engine)
    clients_df_from_db = pd.read_sql('SELECT * FROM clients', engine)

    print("Data successfully loaded from PostgreSQL into DataFrames for analysis.")
    print("\nPayments DataFrame (from DB) - first 5 rows:")
    print(payments_df_from_db.head().to_markdown(index=False, numalign="left", stralign="left"))
    print("\nClients DataFrame (from DB) - first 5 rows:")
    print(clients_df_from_db.head().to_markdown(index=False, numalign="left", stralign="left"))

except Exception as e:
    print(f"Error loading data from database: {e}")
finally:
    engine.dispose()

Data successfully loaded from PostgreSQL into DataFrames for analysis.

Payments DataFrame (from DB) - first 5 rows:
| transaction_id   | contract_id   | client_id   | transaction_date    | payment_amt   | payment_code   |
|:-----------------|:--------------|:------------|:--------------------|:--------------|:---------------|
| 20175            | 927           | 1           | 2018-05-22 18:08:31 | 66.66         | PAYMENT        |
| 8485             | 927           | 1           | 2017-11-26 17:08:15 | 66.66         | PAYMENT        |
| 13778            | 927           | 1           | 2018-02-22 17:08:23 | 66.66         | PAYMENT        |
| 22768            | 927           | 1           | 2018-06-24 18:08:44 | 66.66         | PAYMENT        |
| 15698            | 927           | 1           | 2018-03-22 17:08:24 | 66.66         | PAYMENT        |

Clients DataFrame (from DB) - first 5 rows:
| client_id   | entity_type                | entity_year_established   |
|:------------|:-------

Check Data is Clean

In [ ]:
total_payments = payments_df_from_db.shape[0]
total_clients = clients_df_from_db.shape[0]

# Payments DataFrame: Check and list duplicate Transaction IDs
print("\nChecking for duplicate Transaction IDs in payments data:")
num_payments_duplicates = payments_df_from_db['transaction_id'].duplicated().sum()
print(f"Number of duplicate Transaction IDs found: {num_payments_duplicates} out of {total_payments}")

if num_payments_duplicates > 0:
    # Get all rows where transaction_id is duplicated
    duplicate_payments = payments_df_from_db[payments_df_from_db['transaction_id'].duplicated(keep=False)]
    print("\nDuplicate Transaction ID rows in Payments DataFrame:")
    # Sort by transaction_id to see duplicates grouped together
    print(duplicate_payments.sort_values(by='transaction_id').to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("No duplicate Transaction IDs found.")

# Clients DataFrame: Check and list duplicate Client IDs
print("\nChecking for duplicate Client IDs in clients data:")
num_clients_duplicates = clients_df_from_db['client_id'].duplicated().sum()
print(f"Number of duplicate Client IDs found: {num_clients_duplicates} out of {total_clients}")

if num_clients_duplicates > 0:
    # Get all rows where client_id is duplicated
    duplicate_clients = clients_df_from_db[clients_df_from_db['client_id'].duplicated(keep=False)]
    print("\nDuplicate Client ID rows in Clients DataFrame:")
    # Sort by client_id to see duplicates grouped together
    print(duplicate_clients.sort_values(by='client_id').to_markdown(index=False, numalign="left", stralign="left"))

    print("\n--- Payments Associated with Duplicate Client IDs ---")
    # Get the unique client_ids that are duplicated
    duplicated_client_ids = duplicate_clients['client_id'].unique()

    if len(duplicated_client_ids) > 0:
        # Filter the payments_df_from_db to find all payments for these client IDs
        payments_for_duplicate_clients = payments_df_from_db[
            payments_df_from_db['client_id'].isin(duplicated_client_ids)
        ]
        print(f"Found {len(payments_for_duplicate_clients)} out of {total_payments} payments associated with {len(duplicated_client_ids)} duplicate client(s)")
        print("\nAll Payments associated with Duplicate Client IDs:")
        # Sort by client_id and then transaction_date for better readability
        print(payments_for_duplicate_clients.sort_values(by=['client_id', 'transaction_date']).to_markdown(index=False, numalign="left", stralign="left"))
    else:
        print("No payments found for the identified duplicate client IDs (this case should ideally not happen if num_clients_duplicates > 0).")
else:
    print("No duplicate Client IDs found.")

print("\n--- Duplicate Check Complete ---")


Checking for duplicate Transaction IDs in payments data:
Number of duplicate Transaction IDs found: 0 out of 25559
No duplicate Transaction IDs found.

Checking for duplicate Client IDs in clients data:
Number of duplicate Client IDs found: 6 out of 1287

Duplicate Client ID rows in Clients DataFrame:
| client_id   | entity_type                    | entity_year_established   |
|:------------|:-------------------------------|:--------------------------|
| 165         | Individual/Sole Trader         | 2006                      |
| 165         | Australian Private Company     | 2015                      |
| 473         | Individual/Sole Trader         | 2008                      |
| 473         | Australian Private Company     | 2016                      |
| 591         | Australian Private Company     | 2013                      |
| 591         | Australian Private Company     | 2007                      |
| 591         | Australian Private Company     | 2015                      |
| 7

Since there are no duplicate transactions, to clean the data:
1. deletes all but the oldest client of clients with duplicate client ids
2. if any clients have duplicate ids and ages, delete both of them, and all the transactions associated with that client id even if those clients are the oldest of the clients with the duplicate ids

In [19]:
# --- Helper function to 'keep oldest, delete ties' ---
def clean_duplicates_by_age(df, id_col, age_col, name):
    """
    Cleans duplicates in a DataFrame: keeps the single oldest entry for each ID,
    discards all entries for an ID if there's a tie for the oldest.
    Returns the cleaned DataFrame and a list of IDs whose all entries were discarded.
    """
    # Start with unique IDs
    df_cleaned_initial = df[~df[id_col].duplicated(keep=False)].copy()
    
    # Identify IDs that have duplicates
    duplicated_ids = df[df[id_col].duplicated(keep=False)][id_col].unique()
    
    ids_fully_discarded = []
    
    if len(duplicated_ids) > 0:
        print(f"\nProcessing {len(duplicated_ids)} duplicated {name} IDs...")
        for dupe_id in duplicated_ids:
            entries = df[df[id_col] == dupe_id].copy()
            
            # Find the minimum (oldest) value for the age column
            oldest_age = entries[age_col].min()

            # Filter entries that have this oldest age
            oldest_entries = entries[entries[age_col] == oldest_age]
            
            if len(oldest_entries) == 1:
                # If there's a unique oldest entry, keep it
                df_cleaned_initial = pd.concat([df_cleaned_initial, oldest_entries], ignore_index=True)
            else:
                # If there's a tie for the oldest, discard all entries for this ID
                ids_fully_discarded.append(dupe_id)
                
        print(f"Number of {name} IDs where all tied entries were discarded: {len(ids_fully_discarded)}")
        if ids_fully_discarded:
            print(f"Discarded {name} IDs due to age ties: {ids_fully_discarded}")
    else:
        print(f"No duplicated {name} IDs to process based on age.")
            
    return df_cleaned_initial, ids_fully_discarded

In [21]:
# --- Start Custom Data Cleaning Block ---
print("\n--- Starting Custom Data Cleaning ---")

# Make copies to work with, preserving originals from previous steps
payments_temp_df = payments_df_from_db.copy()
clients_temp_df = clients_df_from_db.copy()

# Ensure 'age' columns are in appropriate formats
clients_temp_df['entity_year_established'] = pd.to_numeric(clients_temp_df['entity_year_established'], errors='coerce')
payments_temp_df['transaction_date'] = pd.to_datetime(payments_temp_df['transaction_date'])

# --- Clean Clients DataFrame based on `client_id` and `entity_year_established` ---
print("\n--- Cleaning Clients ---")
clients_cleaned_df, discarded_client_ids = clean_duplicates_by_age(
    clients_temp_df, 'client_id', 'entity_year_established', 'Client'
)
clients_initial_rows = clients_temp_df.shape[0]
clients_cleaned_rows = clients_cleaned_df.shape[0]
clients_removed_count = clients_initial_rows - clients_cleaned_rows
print(f"Total client entries removed: {clients_removed_count}")
print(f"Clients DataFrame after custom client_id cleaning: {clients_cleaned_df.shape}")


# --- Filter Payments DataFrame based on the cleaned client IDs ---
# This step ensures that all transactions belonging to clients that were completely
# removed in the previous step (due to tied oldest 'entity_year_established') are also removed.
print("\n--- Filtering Payments based on Cleaned Clients ---")
valid_client_ids = clients_cleaned_df['client_id'].unique()
payments_filtered_by_clients_df = payments_temp_df[
    payments_temp_df['client_id'].isin(valid_client_ids)
]
payments_removed_by_client_filter_count = payments_temp_df.shape[0] - payments_filtered_by_clients_df.shape[0]
print(f"Removed {payments_removed_by_client_filter_count} payment transactions associated with discarded client IDs.")
print(f"Payments DataFrame after filtering by valid clients: {payments_filtered_by_clients_df.shape}")
payments_cleaned_df = payments_filtered_by_clients_df.copy()

print("\n--- Data Cleaning Complete ---")
print(f"Original Payments rows: {total_payments} -> Final Cleaned Payments rows: {payments_cleaned_df.shape[0]}")
print(f"Original Clients rows: {total_clients} -> Final Cleaned Clients rows: {clients_cleaned_df.shape[0]}")


# Display the head of the cleaned DataFrames to confirm
print("\nCleaned Payments DataFrame - first 5 rows:")
print(payments_cleaned_df.head().to_markdown(index=False, numalign="left", stralign="left"))

print("\nCleaned Clients DataFrame - first 5 rows:")
print(clients_cleaned_df.head().to_markdown(index=False, numalign="left", stralign="left"))

# You can now proceed with your analysis using 'payments_cleaned_df' and 'clients_cleaned_df'
# For example, to merge them for further analysis:
# merged_analysis_df = pd.merge(payments_cleaned_df, clients_cleaned_df, on='client_id', how='left')


--- Starting Custom Data Cleaning ---

--- Cleaning Clients ---

Processing 5 duplicated Client IDs...
Number of Client IDs where all tied entries were discarded: 0
Total client entries removed: 6
Clients DataFrame after custom client_id cleaning: (1281, 3)

--- Filtering Payments based on Cleaned Clients ---
Removed 0 payment transactions associated with discarded client IDs.
Payments DataFrame after filtering by valid clients: (25559, 6)

--- Data Cleaning Complete ---
Original Payments rows: 25559 -> Final Cleaned Payments rows: 25559
Original Clients rows: 1287 -> Final Cleaned Clients rows: 1281

Cleaned Payments DataFrame - first 5 rows:
| transaction_id   | contract_id   | client_id   | transaction_date    | payment_amt   | payment_code   |
|:-----------------|:--------------|:------------|:--------------------|:--------------|:---------------|
| 20175            | 927           | 1           | 2018-05-22 18:08:31 | 66.66         | PAYMENT        |
| 8485             | 927     